In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import scipy.stats as stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import cross_validation
from sklearn import preprocessing
path='D:/Project Data/Happy_Customer_Bank/'
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline
import math

In [2]:
train=pd.read_csv(path+'Train_nyOWmfK.csv',encoding='iso-8859-1')
testdata=pd.read_csv(path+'Test_bCtAN1w.csv',encoding='iso-8859-1')
train.ix[train['Gender']=='Female','Gender_']=1
train.ix[train['Gender']=='Male','Gender_']=0
train.ix[train['Mobile_Verified']=='N','Mobile_Verified_']=0
train.ix[train['Mobile_Verified']=='Y','Mobile_Verified_']=1
train.ix[train['Filled_Form']=='Y','Filled_Form_']=1
train.ix[train['Filled_Form']=='N','Filled_Form_']=0
train.ix[train['Device_Type']=='Mobile','Device_Type_']=0
train.ix[train['Device_Type']=='Web-browser','Device_Type_']=1

train['age']=[(2015-(1900+int(i[-2:]))) for i in train['DOB'].values]  

train['Loan_Amount_Applied']=train['Loan_Amount_Applied'].fillna(train['Loan_Amount_Applied'].mean())

train['Loan_Tenure_Applied']=train['Loan_Tenure_Applied'].fillna(train['Loan_Tenure_Applied'].median())

train['Existing_EMI']=train['Existing_EMI'].fillna(0)

train['Loan_Amount_Submitted']=train['Loan_Amount_Submitted'].fillna(train['Loan_Amount_Submitted'].mean())

train['Loan_Tenure_Submitted']=train['Loan_Tenure_Submitted'].fillna(train['Loan_Tenure_Submitted'].median())

train['Interest_Rate']=train['Interest_Rate'].fillna(train['Interest_Rate'].median())

train['Processing_Fee']=train['Processing_Fee'].fillna(train['Processing_Fee'].mean())

train['EMI_Loan_Submitted']=train['EMI_Loan_Submitted'].fillna(train['EMI_Loan_Submitted'].mean())

Loan_Amount_Applied_mean=train['Loan_Amount_Applied'].mean()

Loan_Tenure_Applied_median=train['Loan_Tenure_Applied'].median()

Loan_Amount_Submitted_mean=train['Loan_Amount_Submitted'].mean()

Loan_Tenure_Submitted_median=train['Loan_Tenure_Submitted'].median()

Interest_Rate_median=train['Interest_Rate'].median()

Processing_Fee_mean=train['Processing_Fee'].mean()

EMI_Loan_Submitted_mean=train['EMI_Loan_Submitted'].mean()

In [3]:
print(np.unique(train['Source']))

print(np.unique(train['Var1']))

['Other' 'S122' 'S127' 'S133' 'S134' 'S137' 'S143' 'S159' 'S161']
['HAXB' 'HAXF' 'HAZD' 'HBXA' 'HBXB' 'HBXX' 'HCXF']


In [4]:
print(train.shape)
train=train[(train['age'] < 60) | (train['Disbursed'] >=1)]
train=train[(train['Interest_Rate'] < 30) | (train['Disbursed'] >=1) ]
train=train[(train['Loan_Tenure_Applied'] > 1) | (train['Disbursed'] >=1) ]
train=train[(train['Monthly_Income'] < 600000) | (train['Monthly_Income'] > 6000)]
train=train[(train['Existing_EMI'] <= 0) | (train['Existing_EMI'] >= 1000) | (train['Disbursed'] >=1) ]
print(train.shape)
np.unique(train['Disbursed'],return_counts=True)

(70671, 31)
(38019, 31)


(array([0, 1], dtype=int64), array([36777,  1242], dtype=int64))

In [5]:
train.corr()['Disbursed'].order(ascending=False)

Disbursed                1.000000
LoggedIn                 0.801020
Var5                     0.123644
Filled_Form_             0.093556
EMI_Loan_Submitted       0.083339
Mobile_Verified_         0.082513
Loan_Amount_Submitted    0.076521
Processing_Fee           0.057081
age                      0.044139
Existing_EMI             0.011611
Monthly_Income           0.004627
Loan_Tenure_Submitted    0.003144
Var4                    -0.008858
Loan_Amount_Applied     -0.017654
Gender_                 -0.083118
Interest_Rate           -0.102902
Loan_Tenure_Applied     -0.146883
Device_Type_            -0.428315
Name: Disbursed, dtype: float64

In [6]:
Cat_Var=['Var4','Source','Var2','Var1']
Num_Var=['Monthly_Income',
    #'Loan_Amount_Applied',
    #'Existing_EMI',
    'Var5',
         #'Loan_Amount_Submitted',
    #'Loan_Tenure_Submitted',
     # 'Interest_Rate',
    #'Processing_Fee',
    #'EMI_Loan_Submitted'
        ]
Processed_Var=['Gender_',
    'Mobile_Verified_','Filled_Form_',
    'Device_Type_',
    'age']
Target_log=['LoggedIn']
Target=['Disbursed']

In [7]:
Num_Var_log=Num_Var +['LoggedIn']

In [8]:
cat_column=['Var4']
cat_data=train['Var4'].tolist() +testdata['Var4'].tolist()
data_=train[cat_column]
le = preprocessing.LabelEncoder()
le.fit(cat_data)
newcol=le.transform(data_)
enc = OneHotEncoder(sparse=False)
enc.fit(newcol)
print(enc.n_values_)
print(enc.feature_indices_)
dummy_var4=enc.transform(newcol)

cat_column=['Source']
cat_data=train['Source'].tolist() +testdata['Source'].tolist()
data_=train[cat_column]
le1 = preprocessing.LabelEncoder()
le1.fit(cat_data)
newcol=le1.transform(data_)
enc1 = OneHotEncoder(sparse=False)
enc1.fit(newcol)
print(enc1.n_values_)
print(enc1.feature_indices_)
dummy_Source=enc1.transform(newcol)


cat_column=['Var2']
cat_data=train['Var2'].tolist() +testdata['Var2'].tolist()
data_=train[cat_column]
le2 = preprocessing.LabelEncoder()
le2.fit(cat_data)
newcol=le2.transform(data_)
enc2 = OneHotEncoder(sparse=False)
enc2.fit(newcol)
print(enc2.n_values_)
print(enc2.feature_indices_)
dummy_var2=enc2.transform(newcol)

cat_column=['Var1']
cat_data=train['Var1'].tolist() +testdata['Var1'].tolist()
data_=train[cat_column]
le3 = preprocessing.LabelEncoder()
le3.fit(cat_data)
newcol=le3.transform(data_)
enc3 = OneHotEncoder(sparse=False)
enc3.fit(newcol)
print(enc3.n_values_)
print(enc3.feature_indices_)
dummy_var1=enc3.transform(newcol)

[3]
[0 3]
[13]
[ 0 13]
[7]
[0 7]
[7]
[0 7]


In [9]:
#dummy=np.hstack((dummy_var4,dummy_Source,dummy_var2,dummy_var1))
dummy=np.hstack((dummy_var4,dummy_Source,dummy_var2,dummy_var1))
print(dummy.shape)

Num_variable=Num_Var+Processed_Var

train_num=train[Num_variable].as_matrix()
traindata=np.hstack((train_num,dummy))
traindata.shape

(38019, 26)


(38019, 33)

In [10]:
from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(train[Target_log], 2, test_size=.99, random_state=42)
for train_index, test_index in sss:
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_s, X_test_s = traindata[train_index], traindata[test_index]
    y_train_s, y_test_s = train[Target_log].values[train_index], train[Target_log].values[test_index]

X_train_s.shape, X_test_s.shape, y_train_s.shape, y_test_s.shape

y_test_s=[i[0] for i in y_test_s]

# predicting login

In [11]:

rfc = ensemble.RandomForestClassifier()
rfc.fit(X_train_s,y_train_s)
predict_class=rfc.predict(X_test_s)
print(accuracy_score(predict_class, y_test_s))
confusion_matrix(predict_class, y_test_s)
#metrics.auc(predict_class, target_test_1)

0.953080581312


C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  app.launch_new_instance()


array([[35654,  1663],
       [  103,   219]])

In [12]:
logistic = linear_model.LogisticRegression()
logistic.fit(X_train_s,y_train_s)
predict_class=logistic.predict(X_test_s)
print(accuracy_score(predict_class, y_test_s))
confusion_matrix(predict_class, y_test_s)

0.949998671591


C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[35757,  1882],
       [    0,     0]])

In [13]:
bag=ensemble.BaggingClassifier(rfc)
bag.fit(X_train_s,y_train_s)
predict_class2=bag.predict(X_test_s)
print(accuracy_score(predict_class2, y_test_s))
confusion_matrix(predict_class2, y_test_s)


0.954966922607


C:\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:493: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[35736,  1674],
       [   21,   208]])

In [14]:
bag_=ensemble.BaggingClassifier(logistic)
bag_.fit(X_train_s,y_train_s)
predict_class2=bag_.predict(X_test_s)
print(accuracy_score(predict_class2, y_test_s))
confusion_matrix(predict_class2, y_test_s)


0.949998671591


C:\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:493: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[35757,  1882],
       [    0,     0]])

# predicting disb

In [15]:
Numerical_var= Num_Var_log+Processed_Var

train_log=train[Numerical_var].as_matrix()
traindata_log=np.hstack((train_log,dummy))
traindata_log.shape

(38019, 34)

In [16]:
sss1 = StratifiedShuffleSplit(train[Target], 2, test_size=.99, random_state=42)
for train_index1, test_index1 in sss1:
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_log, X_test_log = traindata_log[train_index1], traindata_log[test_index1]
    y_train_log, y_test_log = train[Target].values[train_index1], train[Target].values[test_index1]

In [17]:
rfc_ = ensemble.RandomForestClassifier()
rfc_.fit(X_train_log,y_train_log)
predict_log=rfc_.predict(X_test_log)
print(accuracy_score(predict_log, y_test_log))
confusion_matrix(predict_log, y_test_log)

0.978612609262


C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  from IPython.kernel.zmq import kernelapp as app


array([[36173,   569],
       [  236,   661]])

In [18]:
logistic1 = linear_model.LogisticRegression()
logistic1.fit(X_train_log,y_train_log)
predict_log=logistic1.predict(X_test_log)
print(accuracy_score(predict_log, y_test_log))
confusion_matrix(predict_log, y_test_log)

0.966550652249


C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[36380,  1230],
       [   29,     0]])

In [19]:
bag2=ensemble.BaggingClassifier(rfc_)
bag2.fit(X_train_log,y_train_log)
predict_log_=bag2.predict(X_test_log)
print(accuracy_score(predict_log_, y_test_log))
confusion_matrix(predict_log_, y_test_log)

0.971226653205


C:\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:493: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[36389,  1063],
       [   20,   167]])

In [20]:
bag2_=ensemble.BaggingClassifier(logistic1)
bag2_.fit(X_train_log,y_train_log)
predict_log_=bag2_.predict(X_test_log)
print(accuracy_score(predict_log_, y_test_log))
confusion_matrix(predict_log_, y_test_log)

0.966364674938


C:\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:493: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[36370,  1227],
       [   39,     3]])

In [21]:
name='rf-log-bag-rf-bag-log'

# test datset

In [22]:
testdata.ix[testdata['Gender']=='Female','Gender_']=1
testdata.ix[testdata['Gender']=='Male','Gender_']=0
testdata.ix[testdata['Mobile_Verified']=='N','Mobile_Verified_']=0
testdata.ix[testdata['Mobile_Verified']=='Y','Mobile_Verified_']=1
testdata.ix[testdata['Filled_Form']=='Y','Filled_Form_']=1
testdata.ix[testdata['Filled_Form']=='N','Filled_Form_']=0
testdata.ix[testdata['Device_Type']=='Mobile','Device_Type_']=0
testdata.ix[testdata['Device_Type']=='Web-browser','Device_Type_']=1

testdata['age']=[(2015-(1900+int(i[-2:]))) for i in testdata['DOB'].values]  

testdata['Loan_Amount_Applied']=testdata['Loan_Amount_Applied'].fillna(Loan_Amount_Applied_mean)

testdata['Loan_Tenure_Applied']=testdata['Loan_Tenure_Applied'].fillna(Loan_Tenure_Applied_median)

testdata['Existing_EMI']=testdata['Existing_EMI'].fillna(0)

testdata['Loan_Amount_Submitted']=testdata['Loan_Amount_Submitted'].fillna(Loan_Amount_Submitted_mean)

testdata['Loan_Tenure_Submitted']=testdata['Loan_Tenure_Submitted'].fillna(Loan_Tenure_Submitted_median)

testdata['Interest_Rate']=testdata['Interest_Rate'].fillna(Interest_Rate_median)

testdata['Processing_Fee']=testdata['Processing_Fee'].fillna(Processing_Fee_mean)

testdata['EMI_Loan_Submitted']=testdata['EMI_Loan_Submitted'].fillna(EMI_Loan_Submitted_mean)


In [23]:
cat_column=['Var4']
data_=testdata[cat_column]
newcol=le.transform(data_)
print(enc.n_values_)
print(enc.feature_indices_)
dummy_var4_test=enc.transform(newcol)

cat_column=['Source']
data_=testdata[cat_column]
newcol=le1.transform(data_)
print(enc1.n_values_)
print(enc1.feature_indices_)
dummy_Source_test=enc1.transform(newcol)

cat_column=['Var2']
data_=testdata[cat_column]
newcol=le2.transform(data_)
print(enc2.n_values_)
print(enc2.feature_indices_)
dummy_var2_test=enc2.transform(newcol)

cat_column=['Var1']
data_=testdata[cat_column]
newcol=le3.transform(data_)
print(enc3.n_values_)
print(enc3.feature_indices_)
dummy_var1_test=enc3.transform(newcol)


[3]
[0 3]
[13]
[ 0 13]
[7]
[0 7]
[7]
[0 7]


In [24]:
#dummy_test=np.hstack((dummy_var4_test,dummy_Source_test,dummy_var2_test,dummy_var1_test))
dummy_test=np.hstack((dummy_var4_test,dummy_Source_test,dummy_var2_test,dummy_var1_test))
print(dummy_test.shape)
testdata_num=testdata[Num_variable].as_matrix()
testdata_final=np.hstack((testdata_num,dummy_test))
testdata_final.shape

(37717, 26)


(37717, 33)

In [25]:
predict_class_test=logistic.predict(testdata_final)
np.unique(predict_class_test, return_counts=True)
predict_class_test=[[i] for i in predict_class_test]


predict_ense_test=rfc.predict(testdata_final)
np.unique(predict_ense_test, return_counts=True)
predict_ense_test=[[i] for i in predict_ense_test]

predict_bag_test=bag.predict(testdata_final)
np.unique(predict_bag_test, return_counts=True)
predict_bag_test=[[i] for i in predict_bag_test]

predict_bag2_test=bag_.predict(testdata_final)
np.unique(predict_bag2_test, return_counts=True)
predict_bag2_test=[[i] for i in predict_bag2_test]

In [26]:
testdata_full_final=np.hstack((testdata_final,predict_class_test))
testdata_full_final.shape

predict_log_test=logistic1.predict(testdata_full_final)
predict_log_test=predict_log_test.tolist()
print(np.unique(predict_log_test,return_counts=True))
sub=pd.DataFrame(data={'ID':testdata['ID'],'Disbursed':predict_log_test},columns=['ID','Disbursed'])
#sub.to_csv(path+'sub13.csv',index=False)

(array([0, 1]), array([29350,  8367], dtype=int64))


In [27]:
testdata_full_final=np.hstack((testdata_final,predict_ense_test))
testdata_full_final.shape

predict_log_test_=rfc_.predict(testdata_full_final)
predict_log_test_=predict_log_test_.tolist()
print(np.unique(predict_log_test_,return_counts=True))
sub1=pd.DataFrame(data={'ID':testdata['ID'],'Disbursed':predict_log_test_},columns=['ID','Disbursed'])
#sub1.to_csv(path+'ense_sub13.csv',index=False)

(array([0, 1]), array([37091,   626], dtype=int64))


In [28]:
testdata_full_final=np.hstack((testdata_final,predict_bag_test))
testdata_full_final.shape

predict_log_test_1=bag2.predict(testdata_full_final)
predict_log_test_1=predict_log_test_1.tolist()
print(np.unique(predict_log_test_1,return_counts=True))
sub2=pd.DataFrame(data={'ID':testdata['ID'],'Disbursed':predict_log_test_1},columns=['ID','Disbursed'])
#sub2.to_csv(path+'bag_sub13.csv',index=False)

(array([0, 1]), array([37716,     1], dtype=int64))


In [29]:
testdata_full_final=np.hstack((testdata_final,predict_bag2_test))
testdata_full_final.shape

predict_log_test_1=bag2_.predict(testdata_full_final)
predict_log_test_1=predict_log_test_1.tolist()
print(np.unique(predict_log_test_1,return_counts=True))
sub2_=pd.DataFrame(data={'ID':testdata['ID'],'Disbursed':predict_log_test_1},columns=['ID','Disbursed'])
#sub2.to_csv(path+'bag_sub13.csv',index=False)

(array([0, 1]), array([37252,   465], dtype=int64))


In [30]:
submission=pd.merge(pd.merge(pd.merge(sub,sub1,how='inner', on='ID'),sub2,how='inner', on='ID'),sub2_,how='inner', on='ID')

In [31]:
submission.columns

Index(['ID', 'Disbursed_x', 'Disbursed_y', 'Disbursed_x', 'Disbursed_y'], dtype='object')

In [32]:
submission['max']=submission[['Disbursed_x', 'Disbursed_y', 'Disbursed_x', 'Disbursed_y']].max(axis=1)

In [33]:
submission1=pd.DataFrame(data={'ID': testdata['ID'],'Disbursed':submission['max']},columns=['ID','Disbursed'])
submission1.to_csv(path+name+'.csv',index=False)